# Publications markdown generator

Takes a CSV of publications with metadata and converts them into the format needed for the Hugo website.


In [1]:
import pandas as pd

## Import CSV

Pandas makes this easy with the read_csv function. 

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [101]:
publications = pd.read_csv("publications.csv", header=0, parse_dates=[1], dayfirst=True, na_filter=False)
#publications.head()


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [102]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;",
    ":": "&#58;",
    "[": "&#91;",
    "]": "&#93;",
    "/": "&#47;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [103]:
import os

from pathlib import Path

type_lookup = {
    "none": 0,
    "conference": 1,
    "journal": 2,
    "preprint": 3,
    "report": 4,
    "book": 5,
    "chapter": 6,
    "thesis": 7,
    "patent": 8
}

for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    print(md_filename)
    html_filename = str(item.pub_date) + "-" + item.url_slug
    citation = item.venue + (" " + str(item.details)) if item.details != "nan" else ""
    year = str(item.pub_date.year)

    ## YAML variables    
    md = "---\ntitle: \""   + item.title + '"'
    md += "\nsubtitle: \"\""
    md += "\nsummary: \"\""
    md += "\nauthors: "
    for author in item.authors.split(', '):
        md += "\n- " + author
    md += "\n\n"

    md += "\ntags: []"
    md += "\ncategories: []"
    md += "\ndate: " + str(item.pub_date)
    md += "\npublishDate: " + str(item.pub_date)
    md += "\nfeatured: false"
    md += "\ndraft: false"
    md += "\npublication: '" + html_escape(item.venue) + "'"
    md += f"\npublication_types: [\"{type_lookup[item.type]}\"]" + "\n"
    if item.preprint:
        md += "\nurl_preprint: '" + item.preprint + "'"
    if item.doi:
        md += "\ndoi: '" + item.doi + "'"
    md += "\nabstract: " + html_escape(item.abstract) + "\n"

    md += "\nprojects: []"
    md += "\n---"
    
    md_folder = Path(year + "-" + item.url_slug)
    if not(os.path.exists(md_folder)):
        os.mkdir(md_folder)
    md_filename = "index.md"
    
    with open(md_folder / md_filename, 'wb') as f:
        f.write(md.encode('utf-8'))

2021-06-11 00:00:00-neurips-szep.md
2021-08-23 00:00:00-frontiers-arce.md
2021-07-19 00:00:00-nat-comms-zhang.md
2021-05-11 00:00:00-pnas-westhoek.md
2021-04-13 00:00:00-nat-comms-ho.md
2020-10-15 00:00:00-bmb-krause.md
2020-08-11 00:00:00-nat-comms-grant.md
2020-02-19 00:00:00-nat-comms-nadezhdin.md
2019-09-17 00:00:00-cmsb-spaccasassi.md
2019-06-09 00:00:00-icml-roeder.md
2019-04-03 00:00:00-nat-nano-joesaar.md
2018-10-05 00:00:00-nat-comp-xiang.md
2018-08-15 00:00:00-interface-murphy.md
2018-07-05 00:00:00-frontiers-boulanger.md
2018-06-01 00:00:00-nat-comp-dalchau.md
2018-03-14 00:00:00-interface-smith.md
2017-11-06 00:00:00-nat-chem-zhang.md
2017-10-30 00:00:00-sci-rep-eccleston.md
2017-07-24 00:00:00-nat-nano-chatterjee.md
2017-07-10 00:00:00-frontiers-eccleston.md
2017-01-05 00:00:00-plos-cb-cardelli.md
2016-02-08 00:00:00-sci-rep-cardelli.md
2016-01-26 00:00:00-msb-grant.md
2015-10-20 00:00:00-sci-rep-bailey.md
2015-10-05 00:00:00-acs-sb-rudge.md
2015-07-21 00:00:00-dna-dalchau